In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.datasets import load_digits
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

from utils.nn.layer import Dense
from utils.nn.optimizer import Adam
from utils.nn.neuron import Activation
from utils.nn.sequential import Sequential
from utils.nn.losses import categorical_crossentropy_loss
from utils.nn.initializer import Initializer, InitializationType

In [ ]:
mnist_dataset = load_digits()
mnist_dataset.data

In [ ]:
mnist_dataset.target

In [ ]:
def visualize_samples(X, y_true, y_pred=None, num_samples=16, figsize=(12, 8), title="MNIST Samples"):
    num_samples = min(num_samples, len(X))
    
    # Calculate grid dimensions (roughly square)
    X = np.array(X)

    cols = int(np.ceil(np.sqrt(num_samples)))
    rows = int(np.ceil(num_samples / cols))
    
    fig, axes = plt.subplots(rows, cols, figsize=figsize)
    fig.suptitle(title, fontsize=16)
    
    # Flatten axes array for easier indexing
    axes = axes.flatten() if num_samples > 1 else [axes]
    
    for i in range(num_samples):
        # Reshape flattened image back to 8x8
        image = X[i].reshape(8, 8)
        
        axes[i].imshow(image, cmap='gray')
        axes[i].axis('off')
        
        # Create label text
        label_text = f"True: {y_true[i]}"
        if y_pred is not None:
            label_text += f"\nPred: {y_pred[i]}"
            # Color code: green if correct, red if incorrect
            color = 'green' if y_true[i] == y_pred[i] else 'red'
        else:
            color = 'black'
        
        axes[i].set_title(label_text, fontsize=10, color=color)
    
    # Hide empty subplots
    for i in range(num_samples, len(axes)):
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

def visualize_random_samples(X, y_true, y_pred=None, num_samples=16, seed=42):
    np.random.seed(seed)
    indices = np.random.choice(len(X), size=num_samples, replace=False)
    
    X_sample = X[indices]
    y_true_sample = y_true[indices]
    y_pred_sample = y_pred[indices] if y_pred is not None else None
    
    title = "Random MNIST Samples"
    if y_pred is not None:
        title += " (Green=Correct, Red=Incorrect)"
    
    visualize_samples(X_sample, y_true_sample, y_pred_sample, 
                     num_samples=num_samples, title=title)

def visualize_class_samples(X, y_true, target_class, num_samples=8):
    class_indices = np.where(y_true == target_class)[0]
    if len(class_indices) == 0:
        print(f"No samples found for class {target_class}")
        return
    
    # Select random samples from this class
    selected_indices = np.random.choice(class_indices, 
                                      size=min(num_samples, len(class_indices)), 
                                      replace=False)
    
    X_sample = X[selected_indices]
    y_true_sample = y_true[selected_indices]
    
    visualize_samples(X_sample, y_true_sample, 
                     num_samples=len(selected_indices),
                     title=f"Samples of Digit {target_class}")

# Test the visualization with the loaded data
print(f"Dataset shape: {mnist_dataset.data.shape}")
print(f"Labels shape: {mnist_dataset.target.shape}")
print(f"Unique labels: {np.unique(mnist_dataset.target)}")

# Visualize some random samples
visualize_random_samples(mnist_dataset.data, mnist_dataset.target, num_samples=16)

In [ ]:
X = mnist_dataset.data
y = mnist_dataset.target.reshape(-1, 1)

#Take only first 400 images
X = X[:500]
y = y[:500]


onehot_encoder = OneHotEncoder()

y_onehot = onehot_encoder.fit_transform(y).toarray()



X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.15, random_state=42)
print(f"X_train: {X_train.shape} y_train: {y_train.shape}")
print(f"X_test: {X_test.shape} y_test: {y_test.shape}")


In [ ]:
y_train[:3]

In [ ]:
model = Sequential(
    layers=[
        Dense(
            shape=(64, 32), 
            activation=Activation.RELU,
            initializer=Initializer(fill_type=InitializationType.HE_NORMAL, fan_in=64, fan_out=32)
        ),
        Dense(
            shape=(32, 16),
            activation=Activation.RELU,
            initializer=Initializer(fill_type=InitializationType.HE_NORMAL, fan_in=32, fan_out=16)
        ),
        Dense(
            shape=(16, 10),
            activation=Activation.SOFTMAX,
            initializer=Initializer(fill_type=InitializationType.HE_NORMAL, fan_in=16, fan_out=10)
        )
    ]
)

X_train = X_train.tolist()
X_test = X_test.tolist()
y_train = y_train.tolist()
y_test = y_test.tolist()


In [ ]:
LR=0.01
BATCH_SIZE=16
EPOCHS=100

optimizer = Adam(params=model.parameters(), learning_rate=LR)

model.fit(
    x_train=X_train,
    y_train=y_train,
    optimizer=optimizer,
    loss_func=categorical_crossentropy_loss,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    metric="accuracy",
    x_validate=X_test,
    y_validate=y_test,
    display_interval=1
)

In [ ]:
y_test_predictions = model.forward_batch(X_test)
y_test_prediction_labels = [int(np.argmax(pred)) for pred in y_test_predictions]
y_test_true_labels = [int(np.argmax(yt)) for yt in y_test]

In [ ]:
visualize_samples(X_test, y_true=y_test_true_labels, y_pred=y_test_prediction_labels, title="MNIST Samples with predicions from NN", num_samples=42)